<a href="https://colab.research.google.com/github/EduardoMoraesRitter/Introducao-NLP-analise-sentimento/blob/master/classificao_resumida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tratamento:

remover stopwords
remover pontuacao
remover acentos
colocar em miusculo
colocar em stemmer

In [18]:
import pandas as pd

import nltk
nltk.download('stopwords') #remove palavras curtas
nltk.download('rslp') #raiz das palavras
from nltk import tokenize #separa as palavras
!pip install Unidecode
from unidecode import unidecode #remove acento
from string import punctuation #remove pontuacao

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [5]:
resenha = pd.read_csv('https://raw.githubusercontent.com/EduardoMoraesRitter/Introducao-NLP-analise-sentimento/master/IMDB_KAGGLE.CSV', delimiter=";", encoding='latin1')
resenha

,id,text_pt,sentiment
0,1,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...
49454,49456,"Como a média de votos era muito baixa, e o fat...",pos
49455,49457,O enredo teve algumas reviravoltas infelizes e...,pos
49456,49458,Estou espantado com a forma como este filme e ...,pos
49457,49459,A Christmas Together realmente veio antes do m...,pos


In [0]:
resenha['classificacao'] = resenha['sentiment'].replace(["neg", "pos"], [0,1])

In [0]:
stemmer = nltk.RSLPStemmer()
tokenize_espaco_pontuacao = tokenize.WordPunctTokenizer()

In [20]:
stopwords = nltk.corpus.stopwords.words('portuguese')
pontuacao = list(punctuation)

#juntar com a lista de stop word
pontuacao_stopword = pontuacao + stopwords

pontuacao_stopword_minuscula_sem_acentos_steam = [unidecode(stemmer.stem(texto.lower())) for texto in pontuacao_stopword]

print(pontuacao_stopword_minuscula_sem_acentos_steam)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', 'de', 'a', 'o', 'que', 'e', 'e', 'do', 'da', 'em', 'um', 'par', 'com', 'nao', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'do', 'com', 'mas', 'ao', 'ele', 'da', 'a', 'seu', 'sua', 'ou', 'qu', 'muit', 'no', 'ja', 'eu', 'tambem', 'so', 'pel', 'pel', 'ate', 'iss', 'ela', 'entr', 'depois', 'sem', 'mesm', 'ao', 'seu', 'qu', 'na', 'me', 'ess', 'ele', 'voc', 'ess', 'num', 'nem', 'sua', 'meu', 'as', 'minh', 'num', 'pel', 'ela', 'qual', 'no', 'lhe', 'del', 'ess', 'ess', 'pel', 'est', 'del', 'tu', 'te', 'voc', 'vo', 'lhe', 'meu', 'minh', 'teu', 'tua', 'teu', 'tua', 'noss', 'noss', 'noss', 'noss', 'del', 'del', 'est', 'est', 'est', 'aquel', 'aquel', 'aquel', 'aquel', 'ist', 'aquil', 'est', 'est', 'est', 'est', 'estiv', 'estev', 'estiv', 'estiv', 'est', 'est', 'est', 'estiv', 'estiver', 'estej', 'estej', 'estej', 'estiv', 'estives

In [0]:
frase_processada = list()
for frase in resenha.text_pt:
  nova_frase = list()
  lista_palavras = tokenize_espaco_pontuacao.tokenize(frase)
  for palavra in lista_palavras:
      palavra_minuscula_sem_acentos_steam = unidecode(stemmer.stem(palavra.lower()))
      if palavra_minuscula_sem_acentos_steam not in pontuacao_stopword_minuscula_sem_acentos_steam:
        nova_frase.append(palavra_minuscula_sem_acentos_steam)
  frase_processada.append(' '.join(nova_frase))

resenha["tratado"] = frase_processada

In [27]:
resenha.head()

,id,text_pt,sentiment,classificacao,tratado
0,1,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0,vez sr costn arrum film temp necess alem terr ...
1,2,Este é um exemplo do motivo pelo qual a maiori...,neg,0,exempl motiv maior film aca gener chat nad val...
2,3,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0,prim tud odei rap imbecil pod agir arm pressio...
3,4,Nem mesmo os Beatles puderam escrever músicas ...,neg,0,beatl pud escrev music tod gost emb walt hill ...
4,5,Filmes de fotos de latão não é uma palavra apr...,neg,0,film fot lat palavr apropri verdad tant ous vi...


In [0]:
tfidf = TfidfVectorizer(lowercase=False, ngram_range = (1,2))
regressa_logistica = LogisticRegression()

In [29]:
vetor_tfidf = tfidf.fit_transform(resenha.tratado)

treino, teste, classe_treino, classe_teste = train_test_split(vetor_tfidf, 
                                                              resenha['classificacao'],
                                                              random_state = 42)

regressa_logistica.fit(treino, classe_treino)
regressa_logistica.score(teste, classe_teste)

0.8833805095026284

In [30]:
#olhas os pessoas dos termos que a regrassao logistica ultilizou
pesos = pd.DataFrame(
    regressa_logistica.coef_[0].T,
    index = tfidf.get_feature_names()
)

print(pesos.nlargest(20, 0))
print(pesos.nsmallest(20, 0))

                    0
otim         9.935408
excel        9.114916
perfeit      6.938498
incr         6.893152
maravilh     6.233774
favorit      5.914672
divert       5.524272
ador         5.069533
brilh        4.895002
hoj          4.542456
fant         4.305807
definitiv    4.241743
am           4.230152
recom        4.098604
aind         3.976112
vid          3.975999
agrad        3.971258
surpreend    3.908476
melhor film  3.865690
gost         3.864185
                 0
ruim    -11.772484
pi      -11.524747
horr     -7.906271
nad      -7.688091
chat     -7.494847
parec    -6.978058
nenhum   -6.459585
tent     -5.892379
mal      -5.699906
pi film  -5.615525
lix      -5.610277
estup    -5.446281
rot      -5.381422
ridicul  -5.300826
infeliz  -5.215294
minut    -5.208752
irrit    -5.202232
menos    -5.016487
terr     -4.926150
sid      -4.719496
